In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Reading data set

In [ ]:
df=pd.read_csv("/kaggle/input/deepnlp/Sheet_1.csv")
df.head()

Some variable having NaN values and not important in further analysis.Removing such variable from data using simple drop command.

In [ ]:
df_data=df.drop(["Unnamed: 3","Unnamed: 4","Unnamed: 5","Unnamed: 6","Unnamed: 7"],axis="columns")
df_data.count()

# Word Cloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt
text_df = " ".join(review for review in df_data.response_text)
wordcloud = WordCloud(background_color="red").generate(text_df)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# **Word frequency Counter plot**

In [ ]:
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from yellowbrick.text import FreqDistVisualizer
vectorizer = CountVectorizer(stop_words='english')
docs       = vectorizer.fit_transform(text for text in df_data['response_text'])
features   = vectorizer.get_feature_names()

visualizer = FreqDistVisualizer(
    features=features, size=(1080, 720)
)
visualizer.fit(docs)
visualizer.show()

Obervation
    Word like 
1.     friend
2.     help
3.     talk
4.     going
 are repeated more then other words.
    

In each and every observation stopword are present and that are not importanat of our point of view." from nltk.corpus import stopwords " Library help us to remove stopwords from our data.

In [ ]:
import re
from nltk.corpus import stopwords
stops = stopwords.words('english')

I am using here Stemmer to strim word.Also we can use Tokenizer or Lemmatizer.Some time stemmer strim word but word like "GO","Went","Going" are back to their root meaning.In such case we use Lemmatizer.
Tokenizer split sentence into seperate word.Stemmer remove "ed","tion","Ing" etc in short it try to word should be in it's root meaning by removing this word.

In [ ]:
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 80):
    response_text = re.sub('[^a-zA-Z]', ' ', df_data['response_text'][i])
    response_text = response_text.lower()
    response_text = response_text.split()
    ps = PorterStemmer()
    response_text = [ps.stem(word) for word in response_text if not word in set(stops)]
    response_text = ' '.join(response_text)
    corpus.append(response_text)

Creating data frame of important word from every observation.If some word are reated in any obervaton this feature data frame count it's frequency.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1000)
X = cv.fit_transform(corpus).toarray()

Importing response variable from original data frame.

In [ ]:
y_response = df_data.iloc[:, 1]

But this reponse variable in categorical form i.e.flagged and not_flagged.While doing operation we have need to convert it into binary form.Using Numpy we can convert it into binary.

In [ ]:
y=np.where(y_response=="flagged",0,1)

Train test split.Response variable is binary that why I use stratify=y.

In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.4, 
                                                    random_state=1,
                                                    stratify=y)

Model build using random forest algorithm.Because it gives me best accuracy than other algorithm.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(random_state=1211,
                                  n_estimators=500,oob_score=True)

In [ ]:
model_rf.fit( X_train , y_train )
y_pred_probarf = model_rf.predict_proba(X_test)[:,1]
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_pred_probarf)